# reg_part_B_NN

# Data import libraries

In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.model_selection import train_test_split, KFold, LeaveOneOut
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [3]:
name_data_file="heart_failure_clinical_records_dataset"

data = pd.read_csv(f"../../raw_data/{name_data_file}.csv", na_values=["?"])

In [4]:
data

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,62.0,0,61,1,38,1,155000.00,1.1,143,1,1,270,0
295,55.0,0,1820,0,38,0,270000.00,1.2,139,0,0,271,0
296,45.0,0,2060,1,60,0,742000.00,0.8,138,0,0,278,0
297,45.0,0,2413,0,38,0,140000.00,1.4,140,1,1,280,0


# Baseline Model

In [5]:
X = data.drop("time", axis=1)
y = data["time"]

## Linear Regression applied WRONG

This is wrong because it is using the features to predict value of y. We just want a mean of the vvalues of y

In [36]:
outer_folds_k_1 = 10
inner_folds_k_2 = 10
random_state = 42

CV_outer = KFold(n_splits=outer_folds_k_1, shuffle=True, random_state=random_state) 

fold_results = []  # store per-fold errors
outer_test_mse = []

for outer_train_idx, outer_test_idx in CV_outer.split(X): 
    # KFold object returns indices for train and test sets with shuffling meaning that indices are not sequential, use print to verify
    # .split() returns the indices of the samples for each fold, test and train sets

    X_train_outer, X_test_outer = X.iloc[outer_train_idx], X.iloc[outer_test_idx]
    #print(X_train_outer.shape, X_test_outer.shape)
    #print(outer_train_idx)
    #print(outer_test_idx)
    y_train_outer, y_test_outer = y.iloc[outer_train_idx], y.iloc[outer_test_idx]
    #print(y_train_outer.shape, y_test_outer.shape)

    # let´s start of with the inner cross validation

    CV_inner = KFold(n_splits=inner_folds_k_2, shuffle=True, random_state=random_state)
    inner_mse = []
    for inner_train_idx, inner_test_idx in CV_inner.split(X_train_outer):

        X_train_inner, X_test_inner = X_train_outer.iloc[inner_train_idx], X_train_outer.iloc[inner_test_idx]
        y_train_inner, y_test_inner = y_train_outer.iloc[inner_train_idx], y_train_outer.iloc[inner_test_idx] 

        model = LinearRegression(fit_intercept=True)
        model.fit(X_train_inner, y_train_inner)
        y_test_pred_inner = model.predict(X_test_inner)

        inner_mse.append( ((y_test_inner - y_test_pred_inner)**2))
        print(inner_mse)

        

[11      2888.582977
24      2204.748068
32      1243.804739
45      3434.918333
49      1649.014090
50         5.206426
67        10.752847
87      1900.113114
90      7328.216289
118     1025.209307
130       68.874269
142     6529.076786
153     3101.534065
194    12721.446446
201     1411.176045
203     1136.810572
206      569.958989
207     1135.329503
208      823.313344
212      853.859944
225     1841.958742
244     1425.723426
264     4890.561685
267     7948.195013
291     9035.866603
295     9912.973301
297    13833.136099
Name: time, dtype: float64]
[11      2888.582977
24      2204.748068
32      1243.804739
45      3434.918333
49      1649.014090
50         5.206426
67        10.752847
87      1900.113114
90      7328.216289
118     1025.209307
130       68.874269
142     6529.076786
153     3101.534065
194    12721.446446
201     1411.176045
203     1136.810572
206      569.958989
207     1135.329503
208      823.313344
212      853.859944
225     1841.958742
244     14

## y_train_mean = y_train.mean() CORRECT

In [58]:
outer_folds_k_1 = 10
inner_folds_k_2 = 10
random_state = 42

CV_outer = KFold(n_splits=outer_folds_k_1, shuffle=True, random_state=random_state) 

fold_results = []  # store per-fold errors
outer_test_mse = []
outer_fold_index = 0
inner_fold_index = 0

for outer_train_idx, outer_test_idx in CV_outer.split(X):
    outer_fold_index += 1
    # KFold object returns indices for train and test sets with shuffling meaning that indices are not sequential, use print to verify
    # .split() returns the indices of the samples for each fold, test and train sets

    X_train_outer, X_test_outer = X.iloc[outer_train_idx], X.iloc[outer_test_idx]
    #print(X_train_outer.shape, X_test_outer.shape)
    #print(outer_train_idx)
    #print(outer_test_idx)
    y_train_outer, y_test_outer = y.iloc[outer_train_idx], y.iloc[outer_test_idx]
    #print(y_train_outer.shape, y_test_outer.shape)

    # let´s start of with the inner cross validation

    CV_inner = KFold(n_splits=inner_folds_k_2, shuffle=True, random_state=random_state) # folds for inner cross-validation object
    inner_mse_baseline = [] # store per-fold errors for inner CV
    for inner_train_idx, inner_test_idx in CV_inner.split(X_train_outer): 
        inner_fold_index += 1

        X_train_inner, X_test_inner = X_train_outer.iloc[inner_train_idx], X_train_outer.iloc[inner_test_idx]
        y_train_inner, y_test_inner = y_train_outer.iloc[inner_train_idx], y_train_outer.iloc[inner_test_idx] 

        y_train_mean = y_train_inner.mean() # calculate mean of y in training set
        #print(f"y_train_mean: {y_train_mean}")
        y_test_pred_inner = pd.Series(y_train_mean, index=y_test_inner.index)
        #print(y_test_pred_inner)
        #print("\n")
        #print(y_test_inner)

        inner_mse_baseline.append(mean_squared_error(y_test_inner, y_test_pred_inner)) # calculate MSE for this inner fold, makes mean 
        # value of the differences between predicted and actual y values for the test set

    inner_mse_mean = np.mean(inner_mse_baseline) # average MSE across inner folds
    print(f"For outer fold {outer_fold_index} Mean Inner fold MSE:", inner_mse_mean) # just a print to see progress, in eality we will have to do the cross validation using 3 models
    # and the one who has the lowest inner MSE will be selected for the outer test set evaluation



For outer fold 1 Mean Inner fold MSE: 6068.262610662071
For outer fold 2 Mean Inner fold MSE: 6064.930475785971
For outer fold 3 Mean Inner fold MSE: 5976.123268870377
For outer fold 4 Mean Inner fold MSE: 6117.984225693169
For outer fold 5 Mean Inner fold MSE: 6164.234766059408
For outer fold 6 Mean Inner fold MSE: 5928.042359268602
For outer fold 7 Mean Inner fold MSE: 5910.197012358446
For outer fold 8 Mean Inner fold MSE: 5868.612915368192
For outer fold 9 Mean Inner fold MSE: 6024.998737762518
For outer fold 10 Mean Inner fold MSE: 6037.150942437636
